In [1]:
import functools

import numpy as np
from collections import defaultdict
from scipy.special import expit

import pyspark.sql.functions as psf
import pyspark.sql.functions as F
from pyspark.sql import Window
from pyspark.sql.functions import udf, pandas_udf
from pyspark.sql.functions import col
import pyspark.sql as pysql
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, MapType, ArrayType, FloatType, BooleanType
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from copy import deepcopy
# from operator import add
# from functools import reduce

pd.set_option('display.max_columns', None)
spark.conf.set('spark.sql.session.timeZone', 'UTC')

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
264,application_1653679151702_0266,pyspark,idle,Link,Link,gakumar,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Loop Detection

In [2]:
from collections import defaultdict
import pyspark.sql.functions as F
from pyspark.sql import Window
from pyspark.sql.types import BooleanType, ArrayType, StringType, IntegerType, MapType, FloatType, StructType, StructField


# import networkx as nx


class Cycles:
    def __init__(self, graph):
        self._graph = graph
        self.nodes = self.get_nodes()
        self.nodeMap = {node: i for i, node in enumerate(self.nodes)}
        self.visited = ['NOT_VISITED' for _ in range(len(self.nodes))]  # Initialize all nodes unvisited
        self.stack = []  # Stack to keep track of visited nodes
        self.cycles = []

    def get_nodes(self):
        _nodes = set([])
        for k, v in self._graph.items():
            items = [k] + v
            for item in items:
                _nodes.add(item)
        return list(_nodes)

    def print_cycles(self):
        return self.cycles

    def add_cycle(self, v):
        cycle = list()
        cycle.append(self.nodes[self.stack[-1]])
        i = 1
        while cycle[-1] != v and i < len(self.stack):
            i += 1
            cycle.append(self.nodes[self.stack[-i]])

        self.cycles.append(cycle)

    def dfs(self):
        curr = self.stack[-1]
        if self.nodes[curr] in self._graph:
            for neighbour in self._graph[self.nodes[curr]]:
                to = self.nodeMap[neighbour]
                if self.visited[to] == 'ON_STACK':
                    self.add_cycle(to)
                elif self.visited[to] == 'NOT_VISITED':
                    self.stack.append(to)
                    self.visited[to] = 'ON_STACK'
                    self.dfs()

        self.visited[curr] = 'DONE'
        self.stack.pop()

    def find_cycles(self):
        for i, node in enumerate(self.nodes):
            if self.visited[i] == 'NOT_VISITED':
                self.stack = []
                self.stack.append(i)
                self.visited[i] = 'ON_STACK'
                self.dfs()

        return self.print_cycles()


def get_site_graph(data_frame):
    df_site_adj_list = data_frame.select('siteId', 'src', 'dst') \
        .groupby('siteId', 'src') \
        .agg(F.collect_set(F.col('dst')).alias('dst'))
    df_site_adj_list = df_site_adj_list.groupby("siteId") \
        .agg(F.map_from_arrays(F.collect_list("src"), F.collect_list("dst")).alias("graph"))

    return df_site_adj_list


def _is_cycle(graph, v, done, stack):
    done[v] = True
    stack[v] = True

    for neighbour in graph[v]:
        if neighbour in graph:
            if not done[neighbour]:
                if _is_cycle(graph, neighbour, done, stack):
                    print(f'Part of cycle: {neighbour}')
                    return True
            elif stack[neighbour]:
                print(f'Last neighbour found on stack: {neighbour}')
                return True

    stack[v] = False
    return False


def is_cycle(graph):
    src_vertices = graph.keys()
    done = {k: False for k in src_vertices}
    stack = {k: False for k in src_vertices}

    for node in src_vertices:
        if not done[node]:
            if _is_cycle(graph, node, done, stack):
                return True
    return False


def find_cycles(graph):
    cycles = Cycles(graph)
    return cycles.find_cycles()


# def find_cycles_nx(graph):
#     G = nx.DiGraph(graph)
#     return list(nx.simple_cycles(G))
#
# udf_find_cycles_nx = F.udf(find_cycles_nx, ArrayType(ArrayType(StringType())))


# Added for site coverage caluclation
def reverse_graph(graph):
    k = graph.keys()
    v = graph.values()
    
    rev_graph = defaultdict(list)
    
    for key, value in zip(k, v):
        for val in value:
            rev_graph[val].append(key)
            
    return rev_graph


def _device_tag(graph, v, done):
    done[v] = True

    for neighbour in graph[v]:
        if neighbour in graph:
            if not done[neighbour]:
                _device_tag(graph, neighbour, done)
        else:
            done[neighbour] = True

    return None


def get_path(x):
    init_graph = x[0]
    df_sel_nodes = x[1]
    graph = reverse_graph(init_graph)
    src_vertices = graph.keys()
    
    target_vertices = []
    for values in graph.values():
        for val in values:
            target_vertices.append(val)
            
    vertices = set(src_vertices) | set(target_vertices)
    
    done = {k: False for k in vertices}

    source_macs = set([node.split("__")[0] for node in src_vertices])
    target_macs = set([node.split("__")[0] for node in init_graph.keys()])
    
    starting_nodes = source_macs - target_macs
    
    starting_vertices = [node for node in src_vertices if (node.split("__")[0] in starting_nodes)&(node.split("__")[0] in df_sel_nodes)]
    
    for node in starting_vertices:
        if not done[node]:
            _device_tag(graph, node, done)
    return done


udf_visit_count = F.udf(lambda x: len(set([device.split("__")[0] for device, visited in x.items() if visited == True])), IntegerType())

udf_node_count = F.udf(lambda x: len(set([device.split("__")[0] for device in x.keys()])), IntegerType())

udf_coverage = F.udf(lambda x: float(x[0])/float(x[1]), FloatType())
                                        
udf_get_path = F.udf(get_path, MapType(StringType(), BooleanType()))

udf_find_cycles = F.udf(find_cycles, ArrayType(ArrayType(StringType())))

udf_is_cycle = F.udf(is_cycle, BooleanType())

udf_array_size = F.udf(lambda a: [len(e) for e in a], ArrayType(IntegerType()))

udf_extract_mac = F.udf(lambda a: [s.split('__')[0] for lst in a for s in lst if '__' in s], ArrayType(StringType()))

uplink_match_schema = StructType([
    StructField("match", IntegerType()),  
    StructField("curr_mismatch", IntegerType()),
    StructField("pred_mismatch", IntegerType())
])

@F.udf(returnType=uplink_match_schema)
def uplink_match(uplink_curr, uplink_pred):
    if uplink_curr is None:
        if uplink_pred is None:
            return 0, 0, 0
        else:
            return 0, 0, len(uplink_pred)
    elif len(uplink_curr) == 0:
        if uplink_pred is None:
            return 0, 0, 0
        else:
            return 0, 0, len(uplink_pred)
    elif len(uplink_curr) > 0:
        if uplink_pred is None:
            return 0, len(uplink_curr), 0
        else:
            match = len(set(uplink_curr) & set(uplink_pred))
            curr_mismatch = len(uplink_curr) - match
            pred_mismatch = len(uplink_pred) - match
            return match, curr_mismatch, pred_mismatch



def compute(edges, nodes):

#     df_oc = consistency_check(df_oc, df_new_uplink).persist()

#     old_preds_switches = df_oc.filter(F.size("uplink_combined")>0).count()
#     any_match = df_oc.filter((F.size("uplink_combined")>0)&(F.col("match")>0)).count()
#     total_match = df_oc.filter((F.size("uplink_combined")>0)&(F.col("match")>0)&(F.col("curr_mismatch")==0)&(F.col("pred_mismatch")==0)).count()
#     less_uplink_pred = df_oc.filter((F.size("uplink_combined")>0)&(F.col("match")>0)&(F.col("curr_mismatch")>0)&(F.col("pred_mismatch")==0)).count()
#     more_uplink_pred = df_oc.filter((F.size("uplink_combined")>0)&(F.col("match")>0)&(F.col("curr_mismatch")==0)&(F.col("pred_mismatch")>0)).count()
#     new_uplink_pred = df_oc.filter((~F.col("uplink_new").isNull()) & (F.size("uplink_combined")==0)).count()

#     no_uplink_pred = df_oc.filter((F.col("uplink_new").isNull()) & (F.size("uplink_combined")>0)).count()
#     diff_uplink_pred = df_oc.filter((~F.col("uplink_new").isNull()) & (F.size("uplink_combined")>0) & (F.col("match")==0)).count()
    
#     # Logging Consistency Metrics
#     print('Total switches with existing predictions : {}'.format(old_preds_switches))

#     print('Among existing, any match : {}'.format(any_match))

#     print('Among existing, total match : {}'.format(total_match))

#     print('Among existing, less uplinks predicted : {}'.format(less_uplink_pred))

#     print('Among existing, more uplinks predicted : {}'.format(more_uplink_pred))

#     print('New uplinks predicted : {}'.format(new_uplink_pred))
    
#     print('Among existing, no uplink predicted: {}'.format(no_uplink_pred))
    
#     print('Among existing, different uplink predicted: {}'.format(diff_uplink_pred))

#     # Importing from old predictions where new predictions are unavailable
#     updated_edges = mix_predictions(df_oc, edges).persist()

    active_edges, candidate_node = process_data(edges, nodes)

    active_edges_count = active_edges.count()
    print('Total active_edges_count : {}'.format(active_edges_count))

    site_graph_cycle_true = get_sites_with_cycles(active_edges).persist()

    sites_with_cycles = site_graph_cycle_true.count()
    print('Sites with cycles in device collection(relType==uplink) = {}'.format(sites_with_cycles))

    # For site coverage
    df_cycle_cov = site_graph_cycle_true.join(candidate_node, 
                                    (site_graph_cycle_true.siteId == candidate_node.siteId_copy), 
                                    how="left").drop("siteId_copy")
    df_cov_stats = df_cycle_cov.withColumn("nodes_covered", 
                    udf_get_path(F.struct("graph", "mac_list"))) \
        .withColumn("visited_count", udf_visit_count(F.col("nodes_covered"))) \
        .withColumn("node_count", udf_node_count(F.col("nodes_covered"))) \
        .withColumn("coverage", udf_coverage(F.struct(F.col("visited_count"), F.col("node_count")))) \
        .select(*["siteId", "visited_count", "node_count", "coverage"]).persist()

    top_loop_count = df_cov_stats.filter(F.col("visited_count") == 0).count()
    print('Total sites with top level loops {}'.format(top_loop_count))
    
    stats = df_cov_stats.agg({"visited_count":"sum", "node_count": "sum", "coverage":"mean"}).collect()
    print('Coverage in sites with loops {}'.format(stats[0][0]))

    print('Devices left out in sites with loops {}'.format(stats[0][2] - stats[0][1]))
    
    df_site_graph_all_cycles = site_graph_cycle_true.withColumn('cycles', udf_find_cycles(F.col('graph')))
    # df_site_graph_all_cycles_nx = site_graph_cycle_true.withColumn('cycles', udf_find_cycles_nx(F.col('graph')))
    # df_graph_cycles_nx_pd = df_site_graph_all_cycles_nx.select('siteId', 'cycles').toPandas()

    df_graph_cycles = df_site_graph_all_cycles \
        .select('siteId', 'cycles'). \
        withColumn('cycle_length', udf_array_size(F.col('cycles'))). \
        withColumn('num_cycles', F.size(F.col('cycle_length')))

    total_cycles = df_graph_cycles.select(
        F.sum(F.col('num_cycles')).alias('total_cycles')
    ).rdd.map(lambda x: x.asDict()).first()
    if total_cycles:
        print('new_model_total_cycles: ', total_cycles)

    df_device_macs = df_site_graph_all_cycles.select('siteId', 'cycles') \
        .withColumn('macs', udf_extract_mac(F.col('cycles')))
    df_macs = df_device_macs.select(F.explode(F.col('macs')).alias('mac')).distinct()

    total_device_count = df_macs.count()
    print('Total device involved in loops {}'.format(total_device_count))

    df_combined_nodes = df_macs.join(nodes, df_macs.mac == nodes.mac)

    device_types = df_combined_nodes.groupBy('deviceType') \
        .count().rdd.map(lambda x: x.asDict()).collect()
    device_types = {r.get('deviceType'): r.get('count') for r in device_types}
    if device_types:
        print('Total device_types {}'.format(device_types))
    
    return site_graph_cycle_true

def process_data(df_edges, df_nodes):
    """
    * Filter out deviceType=='ap'
    * create src, dst columns with mapping
        src-->(source,sourcePort)
        dst-->(target,targetPort)
    """
    print('Pre-processing node and edge data')
    df_edges = df_edges.withColumn('isExpired', F.when(F.isnull(F.col('expiredAt')), 'False').otherwise('True'))
    df_active_edges = df_edges.filter(F.col('isExpired') == F.lit(False))
    df_active_edges = df_active_edges.filter(F.col('relType') == 'uplink')

    df_nodes = df_nodes.withColumn('isExpired', F.when(F.isnull(F.col('expiredAt')), 'False').otherwise('True'))
    df_active_nodes = df_nodes.filter(F.col('isExpired') == F.lit(False))
    df_active_nodes = df_active_nodes.withColumnRenamed("siteId", "siteId_copy")

    # For site coverage
    candidate_node = df_active_nodes.filter(F.col("deviceType").isin(['switch', 'gateway', 'third'])) \
        .where(F.col("model") != 'SSR') \
        .where(~((F.isnull(F.col("mac")))|(F.col("mac")==''))) \
        .select(*["siteId_copy", "mac", "deviceType"]).distinct() \

    candidate_node = candidate_node.groupby(F.col("siteId_copy")).agg(F.collect_list(F.col("mac")).alias("mac_list"))

    df_join = df_active_edges.join(df_active_nodes, [df_active_edges.source == df_active_nodes.mac,
                                                     df_active_edges.siteId == df_active_nodes.siteId_copy]) \
        .drop('siteId_copy')

    df_active_nodes = df_active_nodes.withColumn('mac_target', df_active_nodes.mac) \
        .withColumn('deviceType_target', df_active_nodes.deviceType)

    df_join = df_join.join(df_active_nodes.select('mac_target', 'deviceType_target', 'siteId_copy'),
                           [df_join.target == df_active_nodes.mac_target,
                            df_join.siteId == df_active_nodes.siteId_copy])

    df_active_edges_filtered = df_join.filter((F.col('deviceType') != 'ap') & (F.col('deviceType_target') != 'ap')) \
        .select('siteId', 'source', 'target', 'sourcePort', 'targetPort', 'sourceVendor', 'targetVendor',
                'deviceType',
                'deviceType_target', 'relType')

    df_active_edges_filtered = df_active_edges_filtered.withColumn('src', F.concat_ws("__", F.col('source'),
                                                                                      F.col('sourcePort'))) \
        .withColumn('dst', F.concat_ws("__", F.col('target'), F.col('targetPort')))

    return df_active_edges_filtered, candidate_node

def get_sites_with_cycles(data_frame):
    print('Building graph in get_sites_with_cycles')
    df_site_graph = get_site_graph(data_frame)
    df_site_graph_cycle = df_site_graph.withColumn('cycle_detected', udf_is_cycle(F.col('graph')))
    return df_site_graph_cycle.filter(F.col('cycle_detected') == F.lit(True))

def consistency_check(df_oc, df_new):
    df_new = df_new.filter((F.col("uplink_value")==1) & (F.col("n_ind")==1)).groupby("site_id", "switch_id").agg(
        F.collect_list("name").alias("uplink_new")
    )

    rank_window = Window().partitionBy(F.col('switch_id')).orderBy(F.col("when").desc())
    df_oc = df_oc.withColumn("rank", F.rank().over(rank_window)).filter(F.col("rank")==1)
    df_oc = df_oc.select("site_id", "switch_id", "uplink_combined")

    df_oc = df_oc.join(df_new, ["site_id", "switch_id"], how="outer")
    df_oc = df_oc.withColumn("match", uplink_match("uplink_combined", "uplink_new")) \
        .select('site_id', 'switch_id', 'uplink_combined', 'uplink_new', F.col('match.*'))

    return df_oc

def mix_predictions(df, df_edges):
    df = df.withColumn("mixed_uplink", F.when(F.col("uplink_new").isNull(), 
        F.col("uplink_combined")).otherwise(F.col("uplink_new")))

    df = df.select('site_id', 'switch_id', F.explode('mixed_uplink').alias("name")) \
        .withColumn("uplink_value", F.lit(1)).persist()

    df = df.withColumnRenamed("site_id", "siteId") \
        .withColumnRenamed("switch_id", "source") \
        .withColumnRenamed("name", "sourcePort")

    df_edges = df_edges.join(df, ["siteId", "source", "sourcePort"], how="left")
    df_edges = df_edges.drop("relType")
    df_edges = df_edges.withColumn("relType", F.when(F.col("uplink_value") == 1, 'uplink').otherwise(''))
    return df_edges

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
def get_hr_str(x):
    x_str = str(x)
    if x<10:
        x_str = "0"+x_str
    return x_str

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
date = '2022-06-06'
env = 'production'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
site_dict = {}

for i in range(0, 24):
    hr = get_hr_str(i)
    s3_path = 's3://mist-aggregated-stats-'+env+'/aggregated-stats/graph/snapshots/device-edges/dt='+date+'/hr='+hr+'/*'
    node_s3_path = 's3://mist-aggregated-stats-'+env+'/aggregated-stats/graph/snapshots/device-nodes/dt='+date+'/hr='+hr+'/*'
    
    print(s3_path)
    print(node_s3_path)
    
    print('reading from s3')
    df_edge = spark.read.parquet(s3_path)
    df_node = spark.read.parquet(node_s3_path)
    
    old_sites = compute(df_edge, df_node)
    
    old_site_list = set(
        old_sites.select('siteId').toPandas()['siteId']
    )
    
    site_dict[hr] = old_site_list
    
    print("########################################################################################")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://mist-aggregated-stats-production/aggregated-stats/graph/snapshots/device-edges/dt=2022-06-06/hr=00/*
s3://mist-aggregated-stats-production/aggregated-stats/graph/snapshots/device-nodes/dt=2022-06-06/hr=00/*
reading from s3
Pre-processing node and edge data
Total active_edges_count : 11883
Building graph in get_sites_with_cycles
Sites with cycles in device collection(relType==uplink) = 172
Total sites with top level loops 139
Coverage in sites with loops 0.08020882789305477
Devices left out in sites with loops 939
new_model_total_cycles:  {'total_cycles': 221}
Total device involved in loops 366
Total device_types {'gateway': 15, 'third': 1, 'switch': 350}
########################################################################################
s3://mist-aggregated-stats-production/aggregated-stats/graph/snapshots/device-edges/dt=2022-06-06/hr=01/*
s3://mist-aggregated-stats-production/aggregated-stats/graph/snapshots/device-nodes/dt=2022-06-06/hr=01/*
reading from s3
Pre-processing 

In [11]:
site_list = [site_dict[k] for k in site_dict.keys()]
common_sites = set.intersection(*site_list)
union_sites = set.union(*site_list)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# Common Sites
print(common_sites)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'4e8627d1-72cc-41fa-96e9-a99c60acc604', '72706575-6eec-42c2-80d8-5ec2a27d232b', '1cca5535-2f46-409c-b692-2f6c69cf5e96', 'a985f593-88e6-4e74-b9af-6048a83851ef', 'cd9af06f-58bd-4bca-86f4-8b4504b758e4', '7933e726-0927-440f-b0f6-0dffeb54b8a2', 'db966083-12c3-4c94-acc3-ac5e52ef42dd', 'a6caef99-589b-409d-996b-9d1bbe71e733', 'ba5c1279-11bc-49e5-94d4-4a8bd9973a87', '5d730b5c-65d2-4061-8b45-c3020c36a087', '6647863e-ec58-4390-9c65-a68dcf270d24', '9cdba47d-cc74-4520-bf73-fd776a8788f8', '90c35de6-f43a-4e66-af30-2a9fcec491b0', 'c69b26ea-baf0-4d37-9f8b-11560a0f78ab', 'b6311022-1133-4683-bcdf-8e0fcec52927', 'd72db198-cb28-48d6-b931-dfe5782e8fe6', '3735ed13-1a9d-4b43-af35-a0a9d829b9cc', '7747f55d-0673-45ca-a32c-c97acde1b438', '86766442-a03b-409b-98aa-a77a4684a311', '1f43a318-19ce-4917-9fb7-5418e6ab9296', 'df4e49a3-f9b5-412f-8b86-ec9d63aa3b82', 'f825b52b-87a1-43d1-844d-2e3cec26924d', '02284e73-03b4-420f-b426-b25290cb9e8a', '34d1fe77-f972-4c05-91c0-3eefedd9e55b', '7b2e6f79-8043-4f0a-a264-cfe099172e04',

In [13]:
# Common Sites
print(union_sites)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'72706575-6eec-42c2-80d8-5ec2a27d232b', 'cd9af06f-58bd-4bca-86f4-8b4504b758e4', 'db966083-12c3-4c94-acc3-ac5e52ef42dd', 'a6caef99-589b-409d-996b-9d1bbe71e733', 'c69b26ea-baf0-4d37-9f8b-11560a0f78ab', 'b6311022-1133-4683-bcdf-8e0fcec52927', 'd72db198-cb28-48d6-b931-dfe5782e8fe6', '7747f55d-0673-45ca-a32c-c97acde1b438', '1f43a318-19ce-4917-9fb7-5418e6ab9296', 'f825b52b-87a1-43d1-844d-2e3cec26924d', '02284e73-03b4-420f-b426-b25290cb9e8a', 'b39dd655-6d59-4050-8aca-c6985276e8df', '218aa8e0-5d72-4d09-ac08-b966956e10ca', '061f9433-7f3a-44f0-8344-542a2e373f62', 'd53c6e86-bc7c-4936-906f-38db5aec207d', 'ea92e8fe-542b-44b5-bab8-d215cd13e82a', '006be760-099e-4df3-b935-c1f547e85a5f', '746a4662-931a-4cdf-b512-13c38e1a2aee', '65727953-f04c-413d-a0c3-92cc202e6128', 'b87125e3-759b-4258-bae3-5e6dc7857d45', '6cf57059-af1c-4bac-a2cc-524144e3d5fc', 'a89fae82-5896-408b-a672-87d7611ecd0f', '20f8ab3e-2bc9-4be3-ac1a-ad89dc28f643', '99becb91-9980-4653-83f0-7a222bad655c', '25181163-8150-4855-939b-949effadf671',

# New Model Predictions

In [4]:
date = '2022-06-13'
env = 'production'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
new_site_dict = {}

for i in [j for j in range(0, 24) if j != 16]:
    hr = get_hr_str(i)
    s3_path = 's3://mist-aggregated-stats-'+env+'/aggregated-stats/graph/snapshots/device-edges/dt='+date+'/hr='+hr+'/*'
    node_s3_path = 's3://mist-aggregated-stats-'+env+'/aggregated-stats/graph/snapshots/device-nodes/dt='+date+'/hr='+hr+'/*'
    
    print(s3_path)
    print(node_s3_path)
    
    print('reading from s3')
    df_edge = spark.read.parquet(s3_path)
    df_node = spark.read.parquet(node_s3_path)
    
    old_sites = compute(df_edge, df_node)
    
    old_site_list = set(
        old_sites.select('siteId').toPandas()['siteId']
    )
    
    new_site_dict[hr] = old_site_list
    
    print("########################################################################################")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://mist-aggregated-stats-production/aggregated-stats/graph/snapshots/device-edges/dt=2022-06-13/hr=00/*
s3://mist-aggregated-stats-production/aggregated-stats/graph/snapshots/device-nodes/dt=2022-06-13/hr=00/*
reading from s3
Pre-processing node and edge data
Total active_edges_count : 133055
Building graph in get_sites_with_cycles
Sites with cycles in device collection(relType==uplink) = 80
Total sites with top level loops 59
Coverage in sites with loops 0.052313974604476246
Devices left out in sites with loops 5959
new_model_total_cycles:  {'total_cycles': 107}
Total device involved in loops 161
Total device_types {'gateway': 14, 'third': 3, 'switch': 145}
########################################################################################
s3://mist-aggregated-stats-production/aggregated-stats/graph/snapshots/device-edges/dt=2022-06-13/hr=01/*
s3://mist-aggregated-stats-production/aggregated-stats/graph/snapshots/device-nodes/dt=2022-06-13/hr=01/*
reading from s3
Pre-processing

In [6]:
new_site_list = [new_site_dict[k] for k in new_site_dict.keys()]
new_common_sites = set.intersection(*new_site_list)
new_union_sites = set.union(*new_site_list)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Common Sites
print(new_common_sites)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'10448f99-05ff-423f-b48e-696413d4c019', '09df1500-f1b3-4501-a805-7af21a1fb90f', '04d8a2d8-4b9c-4314-b1fb-7cec586f437d', 'd7e0e251-82a9-4a84-9622-844ca4fd1e4b', '7933e726-0927-440f-b0f6-0dffeb54b8a2', '4d6abae0-f9eb-40fe-9db2-ccb630d1261b', 'd0508aef-35e8-40cc-a7a1-c62ba148be58', '99becb91-9980-4653-83f0-7a222bad655c', '462562db-5118-4581-bfc7-0fc9d86eafc6', '13cbd57d-9876-4926-80ee-3703970d66fd', '7e9f985d-dfc3-4f23-bf71-33130bd7223f', '8e443f2e-914e-43fc-91e6-0d732c82d610', '2c355e9f-8cdd-4aeb-811e-26907afb3a46', '77a8fec7-dce8-4486-843f-14ec33a99833', 'c6987f0f-c68f-43d9-993e-e9c1d63fc4d7', '54f41c7f-42c2-483c-a073-797964bfffed', '60a31a89-4a68-4a46-a58e-0a8cf866ef7a', '6647863e-ec58-4390-9c65-a68dcf270d24', 'fe45458d-e425-4e65-85b6-1890bcb7ad70', '4747d091-2bda-426d-bb6d-a72b49fb5a33', 'b065a63a-f500-428a-80bf-9bac6301317e', '9c103626-af9d-41f4-ac8c-254cb61863f6', '3604f709-045b-459a-b8df-45a399ab56e4', '128aa802-6601-4197-bb38-c9c2ae2d3994', 'd72db198-cb28-48d6-b931-dfe5782e8fe6',

In [8]:
# Common Sites
print(new_union_sites)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'72706575-6eec-42c2-80d8-5ec2a27d232b', '04d8a2d8-4b9c-4314-b1fb-7cec586f437d', '21cae898-ca82-4a4f-8138-00bca7931f87', '7933e726-0927-440f-b0f6-0dffeb54b8a2', '13cbd57d-9876-4926-80ee-3703970d66fd', '8e443f2e-914e-43fc-91e6-0d732c82d610', 'ba5c1279-11bc-49e5-94d4-4a8bd9973a87', '6647863e-ec58-4390-9c65-a68dcf270d24', '128aa802-6601-4197-bb38-c9c2ae2d3994', 'f8a3c86b-6472-488e-a6c9-ee42d70a6f44', 'cdc7db33-da6e-4186-bd45-95d62f58c9aa', 'd72db198-cb28-48d6-b931-dfe5782e8fe6', '3735ed13-1a9d-4b43-af35-a0a9d829b9cc', '1f43a318-19ce-4917-9fb7-5418e6ab9296', 'd86aaba3-eead-454e-8a83-0131904650b4', 'd8f38c5e-6c95-4dbd-953d-835f5adbf316', '0bc1ffb9-7815-4a4d-ba59-f674dd057d67', '34d1fe77-f972-4c05-91c0-3eefedd9e55b', 'b7294cc0-0c73-4a74-b7f6-761b8fd09948', '2fab5da3-9dab-47f8-9cc9-a76438b5a02e', '10811140-1428-4592-88e5-64ca908a08b2', '766c8be2-dc5c-4e3a-b88d-a14ce6143492', '3057fa2b-e5ee-40fb-a727-82e65a62ae4a', '61236b85-d4d6-4289-b093-3ff48a9adb65', '37af3f6b-7d8c-4605-a804-d603629c2e80',